# Training pipeline for MFCC (train + save)

In [1]:
import sys
import os
from git_root import git_root

import numpy as np
import tensorflow as tf

from tensorflow.keras.optimizers import Adam

sys.path.append(git_root("utils"))
from utils import load_params

from fetch_data import fetch_data_local
from fetch_data import to_numpy_arrays, prepare_tf_dataset

sys.path.append(git_root("models"))
from MFCC_model import setup_model

from validation_utils import plot_history

In [2]:
print(tf.__version__)

2.0.0


In [3]:
params = load_params()

In [69]:
train = fetch_data_local(map_type="mfcc", train=True)
test = fetch_data_local(map_type="mfcc", train=False)

Fetching: data_mfcc_train.json
Fetching: data_mfcc_test.json


In [70]:
print("train")
print(train.shape)
print("test")
print(test.shape)

train
(900, 3)
test
(100, 3)


In [71]:
len(train.iloc[0, 1])

30

In [72]:
train = to_numpy_arrays(train, mfcc=True)
test = to_numpy_arrays(test, mfcc=True)

In [73]:
print("train")
print(len(train[0]))
print(len(train[1]))
print("test")
print(len(test[0]))
print(len(test[1]))

train
900
900
test
100
100


In [74]:
train = prepare_tf_dataset(train[0], train[1])
test = prepare_tf_dataset(test[0], test[1])

In [75]:
tr_sample_batch, tr_label_batch = next(iter(train))
te_sample_batch, te_label_batch = next(iter(test))
print("train")
print(tr_sample_batch.shape, tr_label_batch.shape)
print("test")
print(te_sample_batch.shape, te_label_batch.shape)

train
(32, 40, 50, 30) (32,)
test
(32, 40, 50, 30) (32,)


In [168]:
#We load the model
net = mfcc.setup_model()
net.summary()

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 31, 41, 12)        36012     
_________________________________________________________________
pooling1 (AveragePooling2D)  (None, 15, 20, 12)        0         
_________________________________________________________________
affine (AffineScalar)        (None, 15, 20, 12)        2         
_________________________________________________________________
conv2 (Conv2D)               (None, 13, 18, 12)        1308      
_________________________________________________________________
pooling2 (GlobalAveragePooli (None, 12)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                130       
Total params: 37,452
Trainable params: 37,452
Non-trainable params: 0
_________________________________________________

In [169]:
net.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
history = net.fit(train, epochs=30, validation_data=test)

Epoch 1/500
29/29 [==============================] - 10s 352ms/step - loss: 2.3071 - accuracy: 0.1133 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/500
29/29 [==============================] - 9s 306ms/step - loss: 2.2748 - accuracy: 0.1489 - val_loss: 2.2722 - val_accuracy: 0.1500
Epoch 3/500
29/29 [==============================] - 10s 329ms/step - loss: 2.2450 - accuracy: 0.1567 - val_loss: 2.2427 - val_accuracy: 0.1600
Epoch 4/500
29/29 [==============================] - 9s 299ms/step - loss: 2.2126 - accuracy: 0.1667 - val_loss: 2.2363 - val_accuracy: 0.1600
Epoch 5/500
29/29 [==============================] - 9s 297ms/step - loss: 2.1928 - accuracy: 0.1567 - val_loss: 2.2027 - val_accuracy: 0.1600
Epoch 6/500
29/29 [==============================] - 11s 373ms/step - loss: 2.1780 - accuracy: 0.1567 - val_loss: 2.2225 - val_accuracy: 0.1700
Epoch 7/500
29/29 [==============================] - 8s 275ms/step - loss: 2.1625 - accuracy: 0.1656 - val_loss: 2.1961 - val_accur

In [ ]:
plot_history(history)

In [ ]:
print(f"Final validation accuracy is: {history.history['val_accuracy'][-1]:.3f}")

In [ ]:
#overwrite = False
if overwrite:
    try:
        file_path = git_root("models", "saved_models", "mfcc.h5")
        net.save(file_path)
    except:
        print("Runtime Error: you might want to check the saved model.")
else:
    print("CAN'T OVERWRITE.")